# Дрва на одлучување

In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('UM5c_RGvgQ4')

## Референци:
- [Упатство за инсталирање на потребните библиотеки и екстензии](https://docs.google.com/presentation/d/1tnoeNF_ge9KK_ovTF7xgKVB64ltuV27IPShk3ddSK5w/edit#slide=id.ga20dadbc8e_0_0)- [ID3 алгоритам](https://en.wikipedia.org/wiki/ID3_algorithm)
- [Дрва на одлучување - Википедија](https://en.wikipedia.org/wiki/Decision_tree)
- [Изградба на дрва на одлучување - Википедија](https://en.wikipedia.org/wiki/Decision_tree_learning)
- [Ентропија во теоријата на информации](https://en.wikipedia.org/wiki/Entropy_(information_theory))
- [Информациска придобивка](https://en.wikipedia.org/wiki/Information_gain_in_decision_trees)
- [Библиотека за машинско учење - sklearn](https://scikit-learn.org/)

Дрво на одлучување е граф со дрвенеста структура каде секој јазол е прашање кое го дели на неколку дела даденото множество податоци. Врските во дрвото се одговорите на прашањето поставено од јазолот. Листовите на дрвото се излезите кои ќе ги произведе дрвото за даден влезен податок. 

Дрвото на одлучување ги класифицира дадените влезни податоци така што ќе започне од коренот и одејќи од јазол на јазол ќе стигне до некој лист чија вредност е излез за дадениот влезен податок.

Еве пример за дрво на одлучување кое ќе ни одговори кое превозно средство да го искористиме во зависност од моменталната состојба на денот во кој се наоѓаме.

![Пример за дрво на одлучување](images/decision_tree_example.png)

In [ ]:
import numpy as np
import pandas as pd

Ова дрво го создаваме со помош на податоци со кои располагаме. Нека, на пример, пред нас ја имаме дадената табела од последните 12 дена. Од оваа табела може да биде создадено даденото дрво на одлучување.

In [ ]:
df = pd.DataFrame(
    [['Sun', 44, True, 'Walk'],
     ['Sun', 23, False, 'Bus'],
     ['Sun', 31, True, 'Walk'],
     ['Sun', 7, False, 'Bus'],
     ['Sun', 19, True, 'Bus'],
     ['Cloud', 34, True, 'Walk'],
     ['Cloud', 16, False, 'Bus'],
     ['Cloud', 6, True, 'Walk'],
     ['Cloud', 25, True, 'Walk'],
     ['Cloud', 71, False, 'Bus'],
     ['Rain', 12, False, 'Bus'],
     ['Rain', 34, True, 'Bus']], columns=['Wheather', 'Time', 'Hungry', 'Transport'])
df

## Создавање дрво на одлучување

Основниот алгоритам за создавање дрво на одлучување е [ID3](https://en.wikipedia.org/wiki/ID3_algorithm). Алгоритамот го гради дрвото од коренот кон листовите и притоа користи алчен пристап. 
Накратко, алгоритамот оди вака:
*  Одреди која карактеристика најдобро го дели множеството.
*  Означи ја таа карактеристика како јазол.
*  За секоја можна вредност на одбраната карактеристика ќе создадеме нов јазол кој ќе биде дете на моменталниот јазол.
*  Податоците кои ги имаме за моменталниот јазол ќе ги поделиме на сите негови деца според вредноста на одбраната карактеристика.
*  Ако новокреираните табели со дадените податоци се перфектно класифицирани, запираме, инаку истиот алгоритам го повторуваме за секој новокреиран јазол.

Главно прашање е како да одредиме која карактеристика е најдобра за да го подели множеството. За алгоритамот ID3 најдобра карактеристика е онаа која има најголема [информациска придобивка](https://en.wikipedia.org/wiki/Information_gain_in_decision_trees). Оваа придобивка ја сметаме како мерка која ќе измери колку добро карактеристиката го дели множеството на групи за да можеме успешно да класифицираме. 

Алгоритамот е алчен бидејќи секогаш ја одбира најдобрата поделба само за момнеталната распределба на податоците, а не ја гледа најдобрата можност генерално за целото дрво.

Алгоритамот подразбира дека сите карактеристики се дискретни, а класификацијата е бинарна. 

### Информациска придобивка

Информациската придобивка е статистичка мерка која кажува колку добро карактеристиката го дели множеството на групи за да можеме успешно да класифицираме. На сликата (десно) може да видиме како една карактеристика го дели множеството на два дела. Двата дела имаат приближно еднаков број `+` и `-`. Едноставно, оваа поделба не нѐ води поблиску кон успешна класификација бидејќи подгрупите се хомогени. За разлика од ова, (лево) ја имаме карактеристиката со голема информациска придобивка која подобро го дели множеството бидејќи има изразена нехомогеност. Лесно забележуваме дека во едната подгрупа имаме повеќе `+`, а во другата имаме повеќе `-`. Ова нѐ води кон успешна класификација.

![Висока информациска придобивка](images/high_information_gain.png) 
![Ниска информациска придобивка](images/low_information_gain.png) 

Пред да ја дефинираме точно информациската придобивка, ќе дефинираме што значи [ентропија](https://en.wikipedia.org/wiki/Entropy_(information_theory) во теоријата на инфромации. 

#### Ентропија

Генерално, ентропијата е мерка за неред во едно множество. За нашиот проблем ентропијата ќе ни каже дали во едно многжество податоци, како претходно со `+` и `-`, имаме хомогеност или пак не. Ентропијата $H$ во теоријата на информации ја пресметуваме според равенката на научникот [Клод Шенон](https://en.wikipedia.org/wiki/Claude_Shannon)

$$ H = \sum_i - p_i \log p_i$$

За алгоритамот ID3, неговата бинарна класификација, и дадено множество $S$ каде излезите се означени со `+` и `-`, ентропијата ќе ја запишуваме како

$$ H(S) = - p_+ \log_2 p_+ - p_- \log_2 p_- $$

каде $p_+$ е подмножеството на $S$ каде излезот е `+`, а $p_-$ е подмножеството на $S$ каде излезот е `-`.

Вредноста која ќе ја добиеме е реален број. На пример, за 30 податоци каде 14 се `+`, а 16 се `-`, ќе добиеме дека ентропијата е 0.996. 

In [ ]:
def entropy(a, b):
    """
    Function that calculates information entropy.
    :param a: subset A size
    :param b: subset B size
    :returns: entropy
    """
    if a == 0 or b == 0:
        return 0   
                            #idejata e ako imas 15 plusa i 0 minusi na primer, entropijata e minimalna, neredot e minimalen; taa e max koga imame ednakov broj plusevi i minusi
    m = a + b
    return - a/m * np.log2(a/m) - b/m * np.log2(b/m)

entropy(14, 30)

Во зависност од бројот на `+` и `-` можеме да исцртаме график. Графикот ќе го исцртаме за 100 точки.

In [ ]:
x = np.linspace(start=0, stop=1, num=100)
y = np.array([entropy(p, 1-p) for p in x])

from plotly import graph_objects as go
fig = go.Figure(go.Scatter(x=x, y=y))
fig.update_layout(title='Ентропија за дадено множество', xaxis_title='p+ or p-', yaxis_title='H(S)')

Од тука можеме да видиме дека ентропијата е 0 ако сите податоци од едно множество припаѓаат на една класа. Најголема е кога има ист број податоци од двете класи.

#### Духовита претстава на ентропијата според Стефан :D

Нека имаме $N$ гости. Ги прашуваме кој сака да пие пиво и ги класифицираме гостите во две групи: (1) пие пиво и (2) не пие пиво. Земаме одговор од секој од гостите и одиме во кујната. Таму ни се поставува прашањето "Кој рече дека ќе пие пиво?".
*  Најлесно е да запаметиме ако никој не пие пиво или пак сите ќе пијат.
*  Лесно е да запаметиме ако само еден нарачал пиво. Подеднакво лесно е да запаметиме ако само еден __не__ нерачал пиво.
*  Малку потешко е ако треба да запаметиме дека двајца ќе пијат пиво. Подеднакво тешко е ако треба да запаметиме дека двајца __нема__ да пијат пиво.
*  Уште малку потешко е ако треба да запаметиме дека тројца ќе пијат пиво. Подеднакво тешко е ако треба да запаметиме дека тројца __нема__ да пијат пиво.
*  ...
*  ...
*  ...
*  Најтешко е да запаметиме кои личности ќе пијат, а кои не, ако точно половина пијат, а другата половина не.

Всушност, она што треба да запаметиме е информацијата која ја носи анкетата направена врз гостите. Одговорите на гостите можеме да ги сметаме како стохастички извор на податоци. Ако секоја вечер ја правиме оваа анкета ќе добиеме распределба како на графикот горе. Ова е ентропијата.

Имајќи го предвид овој пример, многу полесно ќе ја разберете првата реченица на википедија која опишува што е [ентропија](https://en.wikipedia.org/wiki/Entropy_(information_theory)), цитирана подолу.

`Information entropy is the average rate at which information is produced by a stochastic source of data.`

Да се вратиме на алгоритамот ID3. Споменавме дека овој алгоритам за најдобра карактеристика ја одбира онаа која има најголема информациска придобивка. По трети пат, таа е мерка која ќе измери колку добро дадена карактеристика $a$ го дели множеството $S$ на групи за да можеме успешно да класифицираме. Генерално, ќе ја изразиме како разлика помеѓу моменталната ентропија на множеството $S$ и ентропијата која ќе ја пресметаме откако карактеристиката $a$ ќе го подели множеството на групи.

$$ IG(S, a) = H(S) - H(S|a) $$

Попрецизно ќе запишеме

$$ IG(S, a) = H(S) - \sum_v \frac{|S_a(v)|}{|S|} H(S_a(v)) $$

Тука $v$ е вредност на карактеристиката $a$ која го дели множеството на повеќе подмножества. Подмножеството на множеството $S$ за вредноста $v$ на карактеристиката $a$ го означуваме како $S_a(v)$. Ентропијата $H(S|a)$ откако $a$ ќе ги подели множеството ја пресметуваме како среднa вредност од ентропиите посебно пресметани за секоја вредност $v$ на карактеристиката $a$. Тие ентропии се помножени со тежински множител $\frac{|S_a(v)|}{|S|}$ кој кажува колкав дел се податоците во $S_a(v)$ од податоците во $S$.

Еве пример за дадено множество со 30 податоци од кои 14 се `+`, а 16 се `-`. Да ја пресметаме информациската придобивка.

![Информациска придобивка](images/information_gain.png)

Левото подмножество има 17 податоци, од кои 13 се `+`, а 4 се `-`. Тежинскиот множител ќе биде $\frac{17}{30}$. Ентропијата ќе ја пресметаме според функцијата `entropy()`со која веќе располагаме.

In [ ]:
h1 = entropy(13, 4)
h1

Десното подмножество има 13 податоци, од кои 1 е `+`, а 12 се `-`. Тежинскиот множител ќе биде $\frac{13}{30}$. Ентропијата ќе ја пресметаме според функцијата `entropy()`со која веќе располагаме.

In [ ]:
h2 = entropy(1, 12)
h2

Ентропијата на дадено множество веќе ја пресметавме погоре, но ќе ја пресметаму пак.

In [ ]:
h_before_split = entropy(14, 16)
h_before_split

Ентропијата $H(S|a)$ ќе ја пресметаме според дадената равенка.

In [ ]:
h_after_split = 17/30 * h1 + 13/30 * h2
h_after_split

За крај, ќе ја пресметаме информациската придобивка. Добиваме дека за оваа поделба на карактеристиката $a$ имаме информациска придобивка од 0.38.

In [ ]:
information_gain = h_before_split - h_after_split
information_gain

Ајде да пресметаме која карактеристика од `wheather`, `time` или пак `hungry` најдобро го дели множеството запишано во променливата `df` на почетокот на оваа дигитална тетратка.

In [ ]:
df

ID3 алгоритамот смета дека карактеристиките имаат дискретни вредности. 
*  `Wheather` има три различни вредности и тоа е во ред. 
*  `Hungry` има две различни вредности и тоа е во ред.
*  `Time` има многу различни вредности. Можеме да ги оставиме така, ама пологично е да направеме како што беше однапред предложено, да поставиме граница од 30 минути и да сметаме за сите податоци дали вреемто е над или под 30 минути. Всушност, така е и конструирано дрвото на одлучување дадено на почетокот на тетратака. Ќе поставиме нова колона Time_over_30 и ќе ја пополниме со `True` или `False`.

In [ ]:
#vo df napravi nova kolona od Time kolonata, vo koja kje stoi dali podatocite ispolnuvaat uslov da imaat vrednost >30 ili ne (VRAKJA TRUE/FALSE)
#najlesno e kompjuterot da raboti so podatoci sto gi razbira odnosno so edinici i so nuli
df['Time_over_30'] = df['Time'] > 30
# оваа редица служи за подредување на колоните бидејќи најлогично е излезната колона Transport да ни е најдесно.
df = df[['Wheather', 'Time', 'Time_over_30', 'Hungry', 'Transport']]
df

Сега ќе дефинираме нова функција која ќе ја пресмета информациската придобивка за дадено множество податоци. Оваа функција ќе ја користиме за пресметка на ентропијата на подмножествата податоци кога некоја карактеристика ќе го подели множеството.

In [ ]:
def entropy_of_dataset(df, output_column, class_1, class_2):
    
#     ги земаме податоците од множеството df каде излезот е еднаков на класа 1
    class_1_data = df.query(f"{output_column} == '{class_1}'")
    
#     ги земаме податоците од множеството df каде излезот е еднаков на класа 2
    class_2_data = df.query(f"{output_column} == '{class_2}'")
    
# ја пресметуваме ентропијата за даденото множество податоци
    return entropy(class_1_data.shape[0], class_2_data.shape[0])


Најдобрата карактеристика е онаа која има најголема информациска придобивка. Ќе ја пресметаме информациската придобивка за сите карактеристики и ќе одлучиме која е најдобра споредувајќи ги. Но, пред тоа треба да ја пресметаме ентропијата на даденото множество податоци.

In [ ]:
output_column = 'Transport'
#tuka sig bi bilo soodvetno i true false za bus/walk da izbereme edno
class_1 = 'Bus'
class_2 = 'Walk'

#se povikuva na pogore definiranata funkcija, koja sto za pratenite parametri celata tabela ja deli na podmnozestva zavisno od nivnoto ispolnuvanje na uslovot class_1 ili class_2
h_before_split = entropy_of_dataset(df, output_column, class_1, class_2)
h_before_split

Карактеристиката `Wheather` има 3 различни вредности.

In [ ]:
#Pravam proverka kakva podelba kje napravi karakteristikata Wheather; znaci sega za sega podatocite ne se podeleni i imaat entropija 0.9 sto e mnogu losa vrednost za entropija, a skoro sovrsena vrednost za informaciska pridobivka


#Tuka samo ja pravam podelbata spored prasanje WHEATHER
sun_data=df.query("Wheather=='Sun'")
cloud_data = df.query("Wheather == 'Cloud'")
rain_data = df.query("Wheather == 'Rain'")


In [ ]:
sun_data

In [ ]:
cloud_data

In [ ]:
rain_data

In [ ]:
#Vo ovoj moment imam 3 tabeli soglasno na toa sto imam 3 razlicni odgovori na prasanjeto Wheather. So ogled na toa sto output-ot od celiot prasalnik mi e Bus/Walk, vo sekoja od trite tabeli proveruvam kakva e raspredelbata 
#na Bus/Walk vo ovie tri tabeli


#pa povikuvam entropija od sun_data, entropija od tabelata cloud_data i entropija od tabelata rain_data vo odnos na izlez Transport i odgovori Bus i Walk


#so ogled na toa sto vo tabela 3 (rain_data) gledam deka imam isklucitelno samo Bus kako odgovor tuka entropijata e entropija(2,0)=0 a ovie 2 i nula gi vlece eve vaka     return entropy(class_1_data.shape[0], class_2_data.shape[0])
def entropy_of_dataset(df, output_column, class_1, class_2):
    
#     ги земаме податоците од множеството df каде излезот е еднаков на класа 1
    class_1_data = df.query(f"{output_column} == '{class_1}'")
    
#     ги земаме податоците од множеството df каде излезот е еднаков на класа 2
    class_2_data = df.query(f"{output_column} == '{class_2}'")
    
# ја пресметуваме ентропијата за даденото множество податоци
    return class_2_data


In [ ]:
h_sun_data=entropy_of_dataset(sun_data, output_column, class_1, class_2)
h_sun_data

In [ ]:
sun_data = df.query("Wheather == 'Sun'")

cloud_data = df.query("Wheather == 'Cloud'")

rain_data = df.query("Wheather == 'Rain'")


h_sun_data = entropy_of_dataset(sun_data, output_column, class_1, class_2)
h_cloud_data = entropy_of_dataset(cloud_data, output_column, class_1, class_2)
h_rain_data = entropy_of_dataset(rain_data, output_column, class_1, class_2)

#print('h_sun_data', h_sun_data)
#print('h_cloud_data', h_cloud_data)
#print('h_rain_data', h_rain_data)
#print()

factor_sun_data = sun_data.shape[0] / df.shape[0]
factor_cloud_data = cloud_data.shape[0] / df.shape[0]
factor_rain_data = rain_data.shape[0] / df.shape[0]

h_after_split_wheather = float(factor_sun_data) * float(h_sun_data) + float(factor_cloud_data) * float(h_cloud_data) + float(factor_rain_data) * float(h_rain_data)
information_gain_wheather = h_before_split - h_after_split_wheather

#print('h_after_split_wheather', h_after_split_wheather)
print('information_gain_wheather', information_gain_wheather)

Карактеристиката `Time_over_30` има 2 различни вредности.

In [ ]:
time_under_30_data = df.query("Time_over_30 == False")
time_under_30_data

In [ ]:
time_over_30_data = df.query("Time_over_30 == True")
time_under_30_data = df.query("Time_over_30 == False")


#entropijata se povikuva na dvete mnozestva, kadesto se pravi proverkata kolku se plusevi a kolku minusi (preku ovaa proverka dali se namalila homogenosta dovolno dobro)
#odnosno dali se iscisteni podatocite so koi raspolagame vo odnos na output-ot
h_time_over_30_data = entropy_of_dataset(time_over_30_data, output_column, class_1, class_2)
h_time_under_30_data = entropy_of_dataset(time_under_30_data, output_column, class_1, class_2)

print('h_time_over_30_data', h_time_over_30_data)
print('h_time_under_30_data', h_time_under_30_data)
print()


#faktorot so koj kje vlijae entropijata za over30 e kolku istanci vleguvaat vo over30 (sto kaj mene e x=((a+)+(b))/vkupniot broj istanci koisto se podeleni niz procesot (sto kaj mene e m)
factor_time_over_30_data = time_over_30_data.shape[0] / df.shape[0]
factor_time_under_30_data = time_under_30_data.shape[0] / df.shape[0]

h_after_split_time_30 = factor_time_over_30_data * h_time_over_30_data + factor_time_under_30_data * h_time_under_30_data
information_gain_time_30 = h_before_split - h_after_split_time_30

print('h_after_split_time_30', h_after_split_time_30)
print('information_gain_time_30', information_gain_time_30)

Карактеристиката `Hungry` има 2 различни вредности.

In [ ]:
hungry_data = df.query("Hungry == True")
hungry_data

In [ ]:
hungry_data = df.query("Hungry == True")
not_hungry_data = df.query("Hungry == False")

h_hungry_data = entropy_of_dataset(hungry_data, output_column, class_1, class_2)
h_not_hungry_data = entropy_of_dataset(not_hungry_data, output_column, class_1, class_2)

print('h_hungry_data', h_hungry_data)
print('h_not_hungry_data', h_not_hungry_data)
print()

factor_hungry_data = hungry_data.shape[0] / df.shape[0]
factor_not_hungry_data = not_hungry_data.shape[0] / df.shape[0]

h_after_split_hungry = factor_hungry_data * h_hungry_data + factor_not_hungry_data * h_not_hungry_data
information_gain_hungry = h_before_split - h_after_split_hungry

print('h_after_split_hungry', h_after_split_hungry)
print('information_gain_hungry', information_gain_hungry)

Заклучуваме дека најголема информациска придобивка има карактеристиката `Hungry`, па таа ја земаме како прво прашање во нашето стебло на одлучување. Понатаму го решаваме истиот проблем посебно за секое подмножество на карактеристиката `Hungry`: `True` или`False` и бараме која карактеристика најдобро ќе ги подели нив.

### Задача за самостојна работа:

- Имплементирајте го алгоритамот ID3 за целото податочно множество и создадете го дрвото.

## Дрва на одлучување преку библиотеката `scikit-learn`

In [ ]:
from sklearn.datasets import load_iris #datasets e modul na bibliotekata scikit-learn kojsto ja dava funkcijata load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data=load_iris()                                #vo data go zapisiuvame vcitanoto mnozestvo podatoci od load_iris()
data

Како податочно множество ќе ги користиме една збирка податоци за перуники. Оваа збирка податоци многу често се користи како множество податоци за испитување на алгоритми за машинско учење, особено за почетници во оваа област. Затоа оваа збирка податоци е дел од библиотеката `scikit-learn`.

![Перуники](images/iris.png)

In [ ]:
data = load_iris()
                                                        #load_iris se sostoi od:
                                                        #data, 
                                                        #target (0-setosa, 1-versicolor, 2-virginica), se sostoi samo od integers
                                                        #target_names (setosa, versicolor, virginica), zacuvani vo lista
                                                        #feature_names (sepal length, sepal width, petal length, petal width) 
                                                        
print('Classes to predict: ', data.target_names)
                                                        #spored pratenite karakteristiki treha da predvidi dali cvekjeto e setosa, versicolor ili virginica 
    

Моментално податоците за наоѓаат во променливата `data` која е збирка од податоци, а ние треба соодветно да ги извадиме податоците.

In [ ]:
x = data.data
y = data.target
print('Number of examples in the data:', x.shape)

Да видиме како изгледаат податоците. Има 4 карактеристики кои се измерени должини на цвеќењето.

In [ ]:
x                                                       #data e struktura sto gi prevzemala site podatoci od load_iris, a vo load_iris data go oznacuva mnozestvoto podatoci-lista od listi 
                                                        #kadesto sekoja od listite ima po 4 podatoci koisto soodvetno oznacuvaat sepal length, sepal width, petal length, petal width

In [ ]:
y                                                       #target e del od load_iris i toj ni dava izlez integer sto ni go oznacuva soodvetniot tip na cvet (spored pratenite karakteristiki)
                                                        #pa taka ako y=0 pristo y=f(x), toa znaci deka listata 4 karakteristiki sto gi imame prateno soodvetstvuvaat na cvetot so oznaka 0 sto e setosa 

Ќе го поделиме множеството на множество за тренирање и множество за тестирање. Ќе тренираме на `X_train` и `y_train`, ќе погодуваме на `X_test`, а резултатите ќе ги споредуваме со `y_test`. Библиотеката има веќе готова функција за делење на множеството на овој начин, а ние само треба да кажеме колкав процент сакаме да биде множество за тренирање, а колкав процент да биде множество за тест.

In [44]:
                                                                                                #znaci se sluzime so funkcijata train_test_split od soodvetnata buiblioteka
                                                                                                #vo dve razlicni listi ni se vlezovite i izlezite 
                                                                                                #x_train i x_test ni se karakteristikite na cvetot. del od tie karakteristiki kje bidat dadeni za treniranje vrz niv, a del za trst
                                                                                                #analogno i za y_train i y_test kadesto ni se izlezite vo odnos na karakteristikite pritoa vnimavaj, sekako mora za sekoj vlez da ima izlezi
                                                                                                #random_state ako go zadademe na fiksna vrednost kje ni garantira deka podatocite kje bidat na isti nacin podeleni sekogas koga kje se izvrsuva kodot
                                                                                                #155 ne vlijae na rezultatite i moze da prima bilo koja nenegativna vrednost
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=155, test_size=0.25)
                                                                                                #test_size kazuva kolkav del od kodot kje bide za testiranje, ostanatiot e za treniranje vrz nego

Го создаваме класификаторот. Аргументите кои можете да ги испратите ќе ги погледнете во документацијата.

In [45]:
clf = DecisionTreeClassifier(criterion = 'entropy')                                             #DecisionTreeClassifier e klasa sto implementira algoritam so drvo na odlucuvanje
                                                                                                #se sostoi od 1.splitting criteria (gini/entrophy, pa bidejkji e predefinirano na gini, morame da pratime kako argument da raboti so entropija)
                                                                                                #rekurzivno delenje, leaf nodes i prediction

Со функцијата `fit()` му ги даваме податоците за тренирање на класификаторот за да тренира на нив. Тој во себе ќе го изгради дрвото на одлучување, а потоа ние ќе можеме д аго предвидиме излезот за нови податоци.
THE IDEA IS TO CREATE A TREE THAT GENERALIZES WELL TO NEW INSTANCES BY EFFECTIVELY CAPTURING THE UNDERLYING PATTERNS IN THE TRAINING DATA!

In [46]:
                                                                                                #na vekje kreiraniot klasifikator mu davame podatoci na koi treba da trenira i nivni soodvetni izlezi                                                                                                #zn
clf.fit(x_train, y_train)
                                                                                                #so izvrsuvanje na ovoj kod vekje klasifikatorot ne e prazen i neistreniran tuku se sostoi od nauceni informacii
                                                                                                #od trening datata i sega vekje moze da se koristi za da pravi pretpostavki na nevideno mnozestvo podatoci

DecisionTreeClassifier(criterion='entropy')

Откако ќе истренира класификаторот, ќе му ги пратиме податоците за тест за тој да ги предвиди.

In [47]:
y_pred =  clf.predict(x_test)                                                                   #na vekje istreniraniot klasifikator, mu prakjame mnozestvo podatoci na koi treba da go predvidi izlezot
                                                                                                #predvideniot izlez e vo y_pred smesten, a nie gi imame tocnite izlezi za pratenite vlezovi vo y_test
                                                                                                #za x_train tocen izlez e y_train
                                                                                                #za x_test tocen izlez e y_test
                                                                                                        #a predviden e y_pred

`y_pred` Се предвидените податоци и треба да ги споредиме со точните. Ќе ја искористиме функцијата `accuracy_score()` за пресметка на точност на две множества податоци. Таа е дел од библиотеката `scikit-learn`.

In [48]:
                                                                                                #slednite redovi kod sluzat za proverka na verodostojnosta na samiot klasifikator
                                                                                                #accuracy_score gi sporeduva pretpostavenite i tocni labels za soodvetni vlezni karakteristiki pa ja vrakja proporcijata
                                                                                                #na pogodeni istanci
            
                                                                                                #opsto accuracy_score(y_true, y_pred)
                
                                                                                                #za da proveri kolku dobro raboti na videni podatoci, za tie podatoci se povikuva predict(x_train)
                                                                                                #pretpostavkite za tie podatoci se vnesuvaat vo y_pred
                                                                                                #a tocnite izlezi za tie podatoci se vo y_train 
print('Accuracy Score on train data: ', accuracy_score(y_true=y_train, y_pred=clf.predict(x_train))) 

                                                                                                #ova ja sledi istata logika samo za nevideni podatoci
                                                                                                #tuka y_pred=clf.predict(x_test), a tocen izlez e y_test
print('Accuracy Score on test data: ', accuracy_score(y_true=y_test, y_pred=y_pred))

Accuracy Score on train data:  1.0
Accuracy Score on test data:  0.9210526315789473


Забележуваме дека точноста на множеството за тренирање е 1, што е својствено за дрвата на одлучување. Точноста на множеството за тест е нешто помала од 1. Ова е сосема логично бидејќи алгоритамот погодува податоци кои никогаш не ги видел.